In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
users = pd.read_csv('users.csv')
users.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,AlexGyver,Alex,NaN,Moscow,alex@alexgyver.ru,NaN,"Инженер, изобретатель, ардуинщик, блогер, люби...",147,3314,0,2016-08-17T22:15:28Z
1,carlcastanas,Carl Andrew Castañas,ICREATECHS,Moscow,NaN,True,NaN,56,2858,3,2021-08-17T07:52:33Z
2,sergeyshaykhullin,Sergey Shaykhullin,NaN,Moscow,NaN,NaN,Kubernetes 👀 .NET 👀 Rust 👀 Next.js,16,1985,3,2019-01-23T16:40:07Z
3,alexey-goloburdin,Alexey Goloburdin,"TO.DIGITAL, SALESBEAT.PRO, DIGITALIZE.TEAM","Moscow, Russia",sterx@rl6.ru,NaN,NaN,27,1853,0,2012-06-26T20:23:25Z
4,richardroberti,Richard Roberti,E CORP,"Moscow, Russia",NaN,NaN,DevOps,4,1779,4,2023-01-27T22:09:59Z


In [3]:
users['hireable'] = users['hireable'].fillna(False).astype(bool)

<ipython-input-3-6ea7a0647cbf>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users['hireable'] = users['hireable'].fillna(False).astype(bool)


In [4]:
top5 = users.sort_values(by='followers', ascending=False).head()
print(','.join(top5['login'].tolist()))

AlexGyver,carlcastanas,sergeyshaykhullin,alexey-goloburdin,richardroberti


In [5]:
users['created_at'] = pd.to_datetime(users['created_at'])

In [6]:
top_earliest = users.sort_values(by='created_at').head()
print(','.join(top_earliest['login'].tolist()))

maxlapshin,veged,alexeyr,alec-c4,alno


In [8]:
repos = pd.read_csv('repositories.csv')
repos.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,AlexGyver,AlexGyver/3dpov,2020-04-18T12:58:52Z,7,7,C++,True,True,mit
1,AlexGyver,AlexGyver/AC_Dimmer,2017-10-04T22:27:47Z,47,47,C,True,True,NaN
2,AlexGyver,AlexGyver/AiFrame,2024-07-17T11:52:58Z,14,14,C,True,True,mit
3,AlexGyver,AlexGyver/Aim-Fan,2018-05-28T13:03:26Z,16,16,C++,True,True,mit
4,AlexGyver,AlexGyver/AlexGyver,2020-10-15T23:12:38Z,50,50,NaN,True,True,NaN


In [11]:
repos['license_name'].value_counts().head(3)

,count
license_name,
mit,4816
other,1267
apache-2.0,1201


In [13]:
users['company'].value_counts().head(1)



,count
company,
YANDEX,22


In [14]:
repos['language'].value_counts().head(1)

,count
language,
JavaScript,2439


In [15]:
users_after_2020 = users[users['created_at'] > '2020-01-01']
users_after_2020.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
1,carlcastanas,Carl Andrew Castañas,ICREATECHS,Moscow,NaN,True,NaN,56,2858,3,2021-08-17 07:52:33+00:00
4,richardroberti,Richard Roberti,E CORP,"Moscow, Russia",NaN,False,DevOps,4,1779,4,2023-01-27 22:09:59+00:00
6,Robi2810,Rodion,SBERBANK,Moscow,NaN,False,DevOps @Sber,31,1675,10242,2022-02-09 14:28:22+00:00
8,PHPTCloud,Yudov Alexei,NaN,Moscow,NaN,False,PHP & Golang software backend develeper,7,1339,1205,2020-04-20 06:17:21+00:00
14,coffinxp,coffin,NaN,Russia (moscow),NaN,False,Security Researcher,11,995,0,2024-07-31 01:54:58+00:00


In [16]:
repos_2020 = repos[repos['login'].isin(users_after_2020['login'].tolist())]
repos_2020['language'].value_counts().head()

,count
language,
JavaScript,222
Python,159
HTML,113
CSS,109
TypeScript,87


In [17]:
avg_stars = repos.groupby('language')['stargazers_count'].mean()
top_lang = avg_stars.idxmax()
top_stars = avg_stars.max()
print(top_lang, top_stars)

Pascal 517.1875


In [18]:
users['leader_strength'] = users['followers'] / (1 + users['following'])
top5_lead = users.sort_values(by='leader_strength', ascending=False).head()
print(','.join(top5_lead['login'].tolist()))

AlexGyver,alexey-goloburdin,yandex,esokolov,yandexdataschool


In [22]:
correlation = users['followers'].corr(users['public_repos'])
correlation

0.05206532928138264

In [20]:
import csv
followers = []
public_repos = []
with open('users.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        followers_count = int(row['followers'])
        public_repos_count = int(row['public_repos'])
        followers.append(followers_count)
        public_repos.append(public_repos_count)
if len(followers) > 1 and len(public_repos) > 1:
    slope, intercept = np.polyfit(public_repos, followers, 1)

    print(f"{slope:.3f}")
else:
    print("Error")

0.213


In [21]:
if repos['has_projects'].dtype == 'object':
    repos['has_projects'] = repos['has_projects'].map({'true': True, 'false': False})
if repos['has_wiki'].dtype == 'object':
    repos['has_wiki'] = repos['has_wiki'].map({'true': True, 'false': False})

correlation = repos['has_projects'].corr(repos['has_wiki'])

print(round(correlation, 3))

0.362


In [24]:
hireable_avg_following = users[users['hireable'] == True]['following'].mean()
non_hireable_avg_following = users[users['hireable'] == False]['following'].mean()
difference = hireable_avg_following - non_hireable_avg_following
difference

-28.727105214667404

In [25]:
from sklearn.linear_model import LinearRegression
users_with_bio = users[(users['bio'].notna()) & (users['bio'] != '')].copy()
users_with_bio.loc[:, 'bio_len'] = users_with_bio['bio'].str.len()

X = users_with_bio['bio_len'].values.reshape(-1,1)
y = users_with_bio['followers']

lr2 = LinearRegression()
lr2.fit(X, y)
lr2.coef_[0]

0.1124359502764165

In [26]:
import csv
from collections import Counter
from datetime import datetime

weekend_repo_counts = Counter()

with open('repositories.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        created_at = row.get('created_at', '')
        if created_at:
            created_date = datetime.fromisoformat(created_at[:-1])

            if created_date.weekday() in [5, 6]:
                user_login = row['login']
                weekend_repo_counts[user_login] += 1

top_users = weekend_repo_counts.most_common(5)

top_logins = [user[0] for user in top_users]

print(','.join(top_logins))

40ants,coagulant,deepfire,kuggaa,hightemp


In [29]:
# Ensure that NaN values in the 'hireable' column are replaced with False
users['hireable'] = users['hireable'].fillna(False)

# Calculate the fraction of users who have provided an email address for hireable=True
fraction_hireable = users[users['hireable'] == True]['email'].notna().mean()

# Calculate the fraction of users who have provided an email address for hireable=False
fraction_non_hireable = users[users['hireable'] == False]['email'].notna().mean()

# Calculate the difference
diff = round(fraction_hireable - fraction_non_hireable, 3)

print(diff)


0.187


In [28]:
new_users = users[users['name'].notna()].copy()
new_users['surname'] = new_users['name'].str.split().str[-1].str.strip()
surname_counts = new_users['surname'].value_counts()
max_count = surname_counts.max()
common_surnames = surname_counts[surname_counts == max_count].index.tolist()
common_surnames.sort()
print(','.join(common_surnames))

Romanov
